In [1]:
import re
from collections import Counter

In [2]:
# function to tokenise words
def words(document):
    "Convert text to lower case and tokenise the document"
    return re.findall(r'\w+', document.lower())

In [3]:
# create a frequency table of all the words of the document
all_words = Counter(words(open('./data/book.txt').read()))

In [4]:
# check frequency of a random word, say, 'chair'
all_words['chair']

64

In [5]:
# look at top 10 frequent words
all_words.most_common(10)

[('the', 34902),
 ('of', 19114),
 ('and', 12735),
 ('to', 10198),
 ('in', 10044),
 ('a', 8599),
 ('it', 4125),
 ('was', 4012),
 ('is', 3912),
 ('that', 3873)]

In [6]:
def edits_one(word):
    "Create all edits that are one edit away from `word`."
    alphabets    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])                   for i in range(len(word) + 1)]
    deletes    = [left + right[1:]                       for left, right in splits if right]
    inserts    = [left + c + right                       for left, right in splits for c in alphabets]
    replaces   = [left + c + right[1:]                   for left, right in splits if right for c in alphabets]
    transposes = [left + right[1] + right[0] + right[2:] for left, right in splits if len(right)>1]
    return set(deletes + inserts + replaces + transposes)

In [7]:
word = "money"
alphabets    = 'abcdefghijklmnopqrstuvwxyz'
splits     = [(word[:i], word[i:])                   for i in range(len(word) + 1)]
deletes    = [left + right[1:]                       for left, right in splits if right]
inserts    = [left + c + right                       for left, right in splits for c in alphabets]
replaces   = [left + c + right[1:]                   for left, right in splits if right for c in alphabets]
transposes = [left + right[1] + right[0] + right[2:] for left, right in splits if len(right)>1]


In [8]:
transposes

['omney', 'mnoey', 'moeny', 'monye']

In [9]:
set(transposes)

{'mnoey', 'moeny', 'monye', 'omney'}

In [10]:
def edits_two(word):
    "Create all edits that are two edits away from `word`."
    return (e2 for e1 in edits_one(word) for e2 in edits_one(e1))

In [11]:
def known(words):
    "The subset of `words` that appear in the `all_words`."
    return set(word for word in words if word in all_words)

In [12]:
def possible_corrections(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits_one(word)) or known(edits_two(word)) or [word])

In [13]:
def prob(word, N=sum(all_words.values())): 
    "Probability of `word`: Number of appearances of 'word' / total number of tokens"
    return all_words[word] / N

In [14]:
print(len(set(edits_one("monney"))))
print(len(set(possible_corrections("emfasize"))))

print(edits_one("monney"))

336
1
{'mbonney', 'monneyf', 'montney', 'moenney', 'monnei', 'monneyk', 'monneay', 'monoey', 'monneny', 'monfney', 'movney', 'moyney', 'ymonney', 'monniey', 'monneuy', 'monnem', 'monnery', 'mrnney', 'monndey', 'mgonney', 'bmonney', 'monne', 'monnay', 'mononey', 'moonney', 'monneyo', 'moqney', 'mohney', 'monnzy', 'smonney', 'monneqy', 'monnqey', 'monnezy', 'mvnney', 'imonney', 'mxonney', 'donney', 'monuney', 'monhey', 'monneyy', 'mooney', 'konney', 'mondey', 'mionney', 'monmney', 'monneyp', 'monnpy', 'oonney', 'monnef', 'zonney', 'monnexy', 'money', 'monyey', 'monnea', 'monnecy', 'monxey', 'monnby', 'monnegy', 'monnej', 'mofnney', 'moeney', 'moiney', 'monnuy', 'mtnney', 'mpnney', 'monmey', 'rmonney', 'maonney', 'lonney', 'mosnney', 'umonney', 'monbney', 'monzney', 'mtonney', 'monnzey', 'monnez', 'mdnney', 'monnvey', 'gonney', 'muonney', 'mohnney', 'monnemy', 'mondney', 'msnney', 'monnsy', 'monwney', 'yonney', 'mzonney', 'monnye', 'nonney', 'momney', 'monned', 'monnkey', 'monpney', 'monn

In [15]:
print(known(edits_one("monney")))

{'money'}


In [16]:
# Let's look at words that are two edits away
print(len(set(edits_two("monney"))))
print(known(edits_one("monney")))

51013
{'money'}


In [17]:
# Let's look at possible corrections of a word
print(possible_corrections("monney"))

{'money'}


In [18]:
# Let's look at probability of a word
print(prob("money"))
print(prob("monkey"))

0.00044745924074735216
0.0


In [19]:
def spell_check(word):
    "Print the most probable spelling correction for `word` out of all the `possible_corrections`"
    correct_word = max(possible_corrections(word), key=prob)
    if correct_word != word:
        return "Did you mean " + correct_word + "?"
    else:
        return "Correct spelling."

In [20]:
# test spell check
print(spell_check("monney"))

Did you mean money?
